In [2]:
import cv2
import json

def draw_knee_keypoints(video_path, json_path, output_path=None):
    # Load JSON annotation data
    with open(json_path, "r") as f:
        data = json.load(f)

    annotations = {ann["image_id"]: ann["keypoints"] for ann in data.get("annotations", [])}
    images = {img["id"]: img["file_name"] for img in data.get("images", [])}

    # Open video
    cap = cv2.VideoCapture(video_path)
    frame_id = 1  # Start frame index
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))  # Get frame rate

    # Define video writer if saving output
    if output_path:
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop when the video ends

        # Get corresponding keypoints from annotation
        if frame_id in annotations:
            keypoints = annotations[frame_id]

            # Extract left knee (index 27) and right knee (index 30) coordinates
            left_knee_x, left_knee_y = int(keypoints[39]), int(keypoints[40])
            right_knee_x, right_knee_y = int(keypoints[42]), int(keypoints[43])

            # Draw keypoints
            cv2.circle(frame, (left_knee_x, left_knee_y), 5, (0, 255, 0), -1)  # Green for left knee
            cv2.circle(frame, (right_knee_x, right_knee_y), 5, (0, 0, 255), -1)  # Red for right knee

            # Add coordinates as text
            cv2.putText(frame, f"L: ({left_knee_x}, {left_knee_y})", (left_knee_x + 20, left_knee_y+10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"R: ({right_knee_x}, {right_knee_y})", (right_knee_x -20, right_knee_y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Show frame
        cv2.imshow("Annotated Video", frame)

        # Save frame if output path is given
        if output_path:
            out.write(frame)

        # Press 'q' to exit
        if cv2.waitKey(25) & 0xFF == ord("q"):
            break

        frame_id += 1  # Move to next frame

    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

# Example Usage:
input = "C:\\awilde\\britta\\LTU\\SkiingProject\\SkiersProject\\Data\\NewData\\Film2025-02-22\\DJI_0087.mp4"
annotation_path = "C:\\Users\\britt\\OneDrive - Luleå University of Technology\\SkiersProject\\Code\\SkiersProject\\data\\alphapose_output\\coco_json\\87.json"
draw_knee_keypoints(input, annotation_path, "output_video.mp4")